<a href="https://colab.research.google.com/github/Sra1panasa/NLP/blob/main/Receipt_FormFilling/concepts_receiptff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Concepts_receipt_FormFilling:**

**Tokenizer**
* A tokenizer is in charge of preparing the inputs for a model. The **library contains tokenizers for all the models**

In [ ]:
!pip install transformers

     |████████████████████████████████| 3.4 MB 6.7 MB/s 
     |████████████████████████████████| 3.3 MB 46.8 MB/s 
     |████████████████████████████████| 895 kB 52.8 MB/s 
     |████████████████████████████████| 596 kB 57.7 MB/s 
     |████████████████████████████████| 61 kB 491 kB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from transformers import BertTokenizerFast
t = BertTokenizerFast.from_pretrained('bert-base-uncased')
print(t)
t.tokenize("hello,hello, i'm testing")

PreTrainedTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})


['hello', ',', 'hello', ',', 'i', "'", 'm', 'testing']

* You can also get a mapping of each token to the respecting word and other things:



In [ ]:
o = t("hello, i'm testing this efauenufefu", add_special_tokens=False, return_attention_mask=False, return_token_type_ids=False)
print(o)
o.words()

{'input_ids': [7592, 1010, 1045, 1005, 1049, 5604, 2023, 1041, 7011, 24997, 16093, 12879, 2226]}


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:336: FutureWarning: `BatchEncoding.words()` property is deprecated and should be replaced with the identical, but more self-explanatory `BatchEncoding.word_ids()` property.
  FutureWarning,


[0, 1, 2, 3, 4, 5, 6, 7, 7, 7, 7, 7, 7]

**Understanding word piece,max length,padding:**
* I am working with Text Classification problem where I want to use the BERT model as the base followed by Dense layers. I want to know how does the 3 arguments work? For example, if I have 3 sentences as:

'My name is slim shade and I am an aspiring AI Engineer',

'I am an aspiring AI Engineer',

'My name is Slim'

* SO what will these 3 arguments do? What I think is as follows:

* **max_length=5-** will keep all the sentences as of length 5 strictly
* **padding=max_length-** will add a padding of 1 to the third sentence
* **truncate=True-** will truncate the first and second sentence so that their length will be strictly 5.

is this correct:


**Ans:**

 * What you have assumed is almost correct, however, there are few differences.

* max_length=5, the max_length specifies the length of the tokenized text. By default, BERT performs word-piece tokenization. For example the word "playing" can be split into "play" and "##ing" (This may not be very precise, but just to help you understand about word-piece tokenization), followed by adding [CLS] token at the beginning of the sentence, and [SEP] token at the end of sentence. Thus, it first tokenizes the sentence, truncates it to max_length-2 (if truncation=True), then prepend [CLS] at the beginning and [SEP] token at the end.(So a total length of max_length)

* padding='max_length', In this example it is not very evident that the 3rd example will be padded, as the length exceeds 5 after appending [CLS] and [SEP] tokens.
* **However, if you have a max_length of 10. The tokenized text corresponds to [101, 2026, 2171, 2003, 11754, 102, 0, 0, 0, 0], where 101 is id of [CLS] and 102 is id of [SEP] tokens. Thus, padded by zeros to make all the text to the length of max_length**

* Likewise, truncate=True will ensure that the max_length is strictly adhered, i.e, longer sentences are truncated to max_length only if truncate=True

In [ ]:
from transformers import BertTokenizerFast
import torch
text=['love','ai']
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

tokens = tokenizer.batch_encode_plus(text,max_length=2,padding='max_length', truncation=True)
print(tokens)
  
text_seq = torch.tensor(tokens['input_ids'])
text_mask = torch.tensor(tokens['attention_mask'])
print(text_seq)
print(text_mask)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

{'input_ids': [[101, 102], [101, 102]], 'token_type_ids': [[0, 0], [0, 0]], 'attention_mask': [[1, 1], [1, 1]]}
tensor([[101, 102],
        [101, 102]])
tensor([[1, 1],
        [1, 1]])


**Trainer and training arguments:**
* We can then fine-tune the model with Trainer

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    evaluation_strategy="epoch",     # Evaluation is done at the end of each epoch.
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    save_total_limit=1,              # limit the total amount of checkpoints. Deletes the older checkpoints.    
)


trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)

trainer.train()


**Understanding token to id,attention mask,tensor:**

**Converting Tokens to IDs:**
* **When the BERT model was trained, each token was given a unique ID. Hence, when we want to use a pre-trained BERT model, we will first need to convert each token in the input sentence into its corresponding unique IDs**

**what is unkwn words and why its giving 100 for those unknown words:**

* if you see below for below all words we are getting same id that is 100 why we are getting 100 for all them ..bcz all of them are unkonw words to bert.
* what is unkwn words lets understand

In [ ]:
from transformers import BertTokenizer
tz = BertTokenizer.from_pretrained("bert-base-cased")
print(tz.convert_tokens_to_ids(["characteristically"]))
print(tz.convert_tokens_to_ids(["sravan"]))
print(tz.convert_tokens_to_ids(["xxxx"]))
print(tz.convert_tokens_to_ids(["iamunknwwordtobert"]))

[100]
[100]
[100]
[100]


* There is an important point to note when we use a pre-trained model. Since the model is pre-trained on a certain corpus, the vocabulary was also fixed. In other words, when we apply a pre-trained model to some other data, it is possible that some tokens in the new data might not appear in the fixed vocabulary of the pre-trained model. This is commonly known as the out-of-vocabulary (OOV) problem.

* **For tokens not appearing in the original vocabulary, it is designed that they should be replaced with a special token [UNK], which stands for unknown token**.

* ***However, converting all unseen tokens into [UNK] will take away a lot of information from the input data. Hence, BERT makes use of a WordPiece algorithm that breaks a word into several subwords, such that commonly seen subwords can also be represented by the model***.

* For example, the word characteristically does not appear in the original vocabulary. Nevertheless, when we use the BERT tokenizer to tokenize a sentence containing this word, we get something as shown below:

In [ ]:
from transformers import BertTokenizer
tz = BertTokenizer.from_pretrained("bert-base-cased")
print(tz.convert_tokens_to_ids(["characteristically"]))
# characteristically taken as unknown word thats why its id given as 100.
# if you obserb aftrer word split/tokenization characterstic has given some id as per the bert vocabulary..

sent = "He remains characteristically confident and optimistic."
tz.tokenize(sent)
['He',
 'remains',
 'characteristic',
 '##ally',
 'confident',
 'and',
 'optimistic',
 '.']

tz.convert_tokens_to_ids(tz.tokenize(sent))
[1124, 2606, 7987, 2716, 9588, 1105, 24876, 119]

[100]


[1124, 2606, 7987, 2716, 9588, 1105, 24876, 119]

* We can see that the word characteristically will be converted to the ID 100, which is the ID of the token [UNK], if we do not apply the tokenization function of the BERT model.

* The BERT tokenization function, on the other hand, will first breaks the word into two subwoards, **namely characteristic and ##ally, where the first token is a more commonly-seen word (prefix) in a corpus, and the second token is prefixed by two hashes ## to indicate that it is a suffix following some other subwords.**

* **After** this tokenization step, all tokens can be converted into their corresponding IDs.

**Summary**
* In summary, an input sentence for a classification task will go through the following steps before being fed into the BERT model.

* 1 Tokenization: breaking down of the sentence into tokens

* 2Adding the [CLS] token at the beginning of the sentence
* 3Adding the [SEP] token at the end of the sentence
* 4Padding the sentence with [PAD] tokens so that the total length equals to the maximum length
* 5Converting each token into their corresponding IDs in the model

* An example of preparing a sentence for input to the BERT model is shown below. For simplicity, we assume the maximum length is 10 in the example below (while in the original model it is set to be 512).

# Original Sentence
Let's learn deep learning!

# Tokenized Sentence
['Let', "'", 's', 'learn', 'deep', 'learning', '!']

# Adding [CLS] and [SEP] Tokens
['[CLS]', 'Let', "'", 's', 'learn', 'deep', 'learning', '!', '[SEP]']

# Padding
['[CLS]', 'Let', "'", 's', 'learn', 'deep', 'learning', '!', '[SEP]', '[PAD]']

# Converting to IDs
[101, 2421, 112, 188, 3858, 1996, 3776, 106, 102, 0]

**Tokenization using the transformers Package**

* While there are quite a number of steps to transform an input sentence into the appropriate representation, we can use the functions provided by the transformers package to help us perform the tokenization and transformation easily. In particular, we can use the function encode_plus, which does the following in one go:

* Tokenize the input sentence
* Add the [CLS] and [SEP] tokens.
* Pad or truncate the sentence to the maximum length allowed
* Encode the tokens into their corresponding IDs Pad or truncate all sentences to the same length.
* **Create the attention masks which explicitly differentiate real tokens from [PAD] tokens**
* The following codes shows how this can be done.

In [ ]:
# Import tokenizer from transformers package
from transformers import BertTokenizer

# Load the tokenizer of the "bert-base-cased" pretrained model
# See https://huggingface.co/transformers/pretrained_models.html for other models
tz = BertTokenizer.from_pretrained("bert-base-cased")

# The senetence to be encoded
sent = "Let's learn deep learning!"

# Encode the sentence
encoded = tz.encode_plus(
    text=sent,  # the sentence to be encoded
    add_special_tokens=True,  # Add [CLS] and [SEP]
    max_length = 64,  # maximum length of a sentence
    pad_to_max_length=True,  # Add [PAD]s
    return_attention_mask = True,  # Generate the attention mask
    return_tensors = 'pt',  # ask the function to return PyTorch tensors
)

# Get the input IDs and attention mask in tensor format
input_ids = encoded['input_ids']
attn_mask = encoded['attention_mask']
#After executing the codes above, we will have the following content for the input_ids and attn_mask variables:


print(input_ids)

print(attn_mask)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


tensor([[ 101, 2421,  112,  188, 3858, 1996, 3776,  106,  102,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2232: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


* The "attention mask" **tells the model which tokens should be attended to and which (the [PAD] tokens) should not (see the documentation for more detail). It will be needed when we feed the input into the BERT model**.

https://albertauyeung.github.io/2020/06/19/bert-tokenization.html/